# ⚛️ H₂ Ground-State VQE — Noisy Simulation

This notebook studies **VQE under realistic noise** via the API:

- `run_vqe_optimizer_comparison`
- `run_vqe`

The goal is to understand how noise affects:

- Optimizer stability  
- Convergence  
- The final **noisy density matrix**  
- Basis-state populations  

Noise model (identical for all optimizers):
- **Depolarizing:** 5%  
- **Amplitude damping:** 5%  

These values are intentionally exaggerated to make behavioural differences clear.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from vqe.core import run_vqe_optimizer_comparison, run_vqe
from vqe.io_utils import IMG_DIR

# Noise settings
depolarizing_prob = 0.05
amplitude_damping_prob = 0.05

seed = 0


# 🔧 Part 1 — Noisy Optimizer Comparison

`run_vqe_optimizer_comparison(...)`

✔ Builds the noisy device  
✔ Runs each optimizer  
✔ Applies both noise channels  
✔ Handles reproducibility + caching  
✔ Saves a convergence plot  
✔ Returns final energies  


In [ ]:
# Optimization settings
optimizers = ["Adam", "GradientDescent", "Momentum", "Nesterov", "RMSProp", "Adagrad"]
steps = 50
stepsize_map = {
    "Adam": 0.2,
    "GradientDescent": 0.05,
    "Momentum": 0.1,
    "Nesterov": 0.1,
    "RMSProp": 0.1,
    "Adagrad": 0.2,
}

comparison = run_vqe_optimizer_comparison(
    molecule="H2",
    ansatz_name="UCCSD",
    optimizers=optimizers,
    steps=steps,
    stepsize=stepsize_map,
    noisy=True,
    depolarizing_prob=depolarizing_prob,
    amplitude_damping_prob=amplitude_damping_prob,
    seed=seed,
    show=True,
    force=False,
)


# ⭐ Part 2 — Identify the Worst Optimizer

Noise flattens the loss landscape, so optimizers behave differently than in
the noiseless case.  

We define the *worst* optimizer as the one with the **highest final energy**.


In [ ]:
worst_optimizer = max(
    comparison["final_energies"],
    key=comparison["final_energies"].get
)

print(f"🏆 Worst optimizer under noise: {worst_optimizer}")


# 🔍 Part 3 — Full Noisy VQE Using the Worst Optimizer

We now run a complete noisy VQE with:

- Ansatz: **UCCSD**  
- Optimizer: worst performer  
- Depolarizing: 10%  
- Amplitude damping: 10%  

`run_vqe(...)` returns energies, metadata, and the full **noisy density matrix**.


In [ ]:
res = run_vqe(
    molecule="H2",
    ansatz_name="UCCSD",
    optimizer_name=worst_optimizer,
    steps=steps,
    stepsize=stepsize_map[worst_optimizer],
    noisy=True,
    depolarizing_prob=depolarizing_prob,
    amplitude_damping_prob=amplitude_damping_prob,
    seed=seed,
    plot=False,
    force=False,
)

rho = np.array(res["final_state_real"]) + 1j * np.array(res["final_state_imag"])
diag = np.diag(rho)

diag


# 🧮 Part 4 — Approximate Ket Representation  

For mixed states, the diagonal entries of ρ correspond to **basis-state
populations**.

We display only those above a small threshold.


In [ ]:
threshold = 1e-2
num_wires = int(np.log2(len(diag)))

terms = [
    f"({amp:.4f}|{idx:0{num_wires}b}⟩)"
    for idx, amp in enumerate(diag)
    if abs(amp) > threshold
]

ket_estimate = " + ".join(terms)

print("Approximate noisy state:")
print(f"|ψ⟩ ≈ {ket_estimate}")


# 📊 Part 5 — Basis-State Population Plot

The bar plot below visualizes **which computational states survive** after noise.


In [ ]:
indices = np.where(abs(diag) > threshold)[0]
vals = diag[indices]
labels = [f"|{i:0{num_wires}b}⟩" for i in indices]

plt.figure(figsize=(10, 5))
plt.bar(labels, vals.real, label="Real")
plt.bar(labels, vals.imag, bottom=vals.real, alpha=0.6, label="Imag")

plt.xlabel("Basis state")
plt.ylabel("Population")
plt.title("H₂ — Noisy VQE Ground State (Diagonal of ρ)")
plt.legend()
plt.tight_layout()

from vqe_qpe_common.plotting import save_plot
save_plot("H2_Noisy_Ground_State.png", kind="vqe", show=False)
plt.show()
